# Reference Data Interface (RDI) Analysis
## Deutsche Börse Market Data Navigation and Security Master Analysis

**Objective**: Demonstrate comprehensive reference data retrieval and analysis for institutional trading applications.

### Business Use Cases
The RDI provides complete security master data required for:
- **Pre-trade Analytics**: Contract specifications, tick sizes, lot sizes, trading hours
- **Risk Management**: Price limits, volatility parameters, position limits
- **Compliance & Surveillance**: Security classifications, regulatory attributes, venue rules
- **Market Microstructure Analysis**: Tick tables, quote size rules, anonymity settings

### Sample Instruments
This analysis focuses on:
1. **EURO STOXX 50 Futures (FESX)** - Market: XEUR, Segment: 675
2. **Deutsche Börse AG (DB1)** - Market: XETR, Segment: 52885, Security: 2504978

In [40]:
import os
import pandas as pd
from dotenv import load_dotenv
from a7 import A7Client
from pprint import pprint

# Load environment variables
load_dotenv()

# Initialize A7 Client
client = A7Client(
    token=os.getenv("A7_API_TOKEN"),
    base_url=os.getenv("A7_BASE_URL", "https://a7.deutsche-boerse.com/api/")
)

print("A7 API Client initialized successfully.")
print(f"Base URL: {client._client.base_url}")

A7 API Client initialized successfully.
Base URL: https://a7.deutsche-boerse.de/api/


## 1. Explore Available Markets

Markets represent the top-level trading venues (e.g., Eurex, Xetra, Frankfurt Stock Exchange).

In [41]:
try:
    markets = client.rdi.get_markets()
    market_list = markets if isinstance(markets, list) else [markets]
    
    print(f"Available Deutsche Börse Markets: {len(market_list)}")
    print("="*60)
    
    # Map market codes to descriptions
    market_descriptions = {
        'XEUR': 'Eurex (Derivatives)',
        'XETR': 'Xetra (Cash Equities)',
        'XFRA': 'Frankfurt Stock Exchange',
        'XEEE': 'EEX (Energy)'
    }
    
    for mic in market_list:
        desc = market_descriptions.get(mic, 'Unknown')
        print(f"  {mic:6s} - {desc}")
    
    print(f"\nTotal venues covered: {len(market_list)}")
    
except Exception as e:
    print(f"ERROR: Failed to retrieve markets - {e}")
    market_list = []

Available Deutsche Börse Markets: 3
  XEEE   - EEX (Energy)
  XETR   - Xetra (Cash Equities)
  XEUR   - Eurex (Derivatives)

Total venues covered: 3


## 2. Explore Market Segments

Market Segments group related instruments within a market. For example:
- XEUR contains segments like FESX (EURO STOXX 50), FDAX (DAX), OGBL (German Bund)
- XETR contains stock segments like DB1 (Deutsche Börse AG), BMW1 (BMW)

In [42]:
# Get market segments for XEUR (Eurex)
target_market = "XEUR"
ref_date = 20251204  # December 4, 2025

try:
    segment_list = client.rdi.get_market_segments(market_id=target_market, ref_date=ref_date)
    
    print(f"\nEurex (XEUR) Market Segments - Reference Date: {ref_date}")
    print("="*60)
    print(f"Total product segments available: {len(segment_list):,}")
    
    # Statistical overview
    print(f"\nSegment ID range: {min(segment_list)} to {max(segment_list)}")
    print(f"Sample segment IDs: {segment_list[:5]}")
    
    # Verify key segments
    key_segments = {
        675: 'FESX (EURO STOXX 50 Futures)',
        688: 'FDAX (DAX Futures)',
        1310: 'OGBL (German Bund Futures)'
    }
    
    print(f"\nKey Product Verification:")
    for seg_id, description in key_segments.items():
        status = "ACTIVE" if seg_id in segment_list else "NOT FOUND"
        print(f"  [{status}] Segment {seg_id:5d}: {description}")
    
except Exception as e:
    print(f"ERROR: Segment retrieval failed - {type(e).__name__}: {e}")
    segment_list = []


Eurex (XEUR) Market Segments - Reference Date: 20251204
Total product segments available: 3,057

Segment ID range: 3 to 1314907
Sample segment IDs: [3, 4, 10, 12, 14]

Key Product Verification:
  [ACTIVE] Segment   675: FESX (EURO STOXX 50 Futures)
  [ACTIVE] Segment   688: FDAX (DAX Futures)
  [NOT FOUND] Segment  1310: OGBL (German Bund Futures)


In [43]:
# Get market segments for XETR (Xetra)
target_market_2 = "XETR"

try:
    segment_list_2 = client.rdi.get_market_segments(market_id=target_market_2, ref_date=ref_date)
    
    print(f"\nXetra (XETR) Cash Equity Segments - Reference Date: {ref_date}")
    print("="*60)
    print(f"Total equity segments: {len(segment_list_2):,}")
    print(f"Coverage: DAX, MDAX, SDAX, TecDAX constituents plus ~3,900 other equities")
    
    # Verify specific securities
    dax_samples = {
        52885: 'DB1 (Deutsche Börse AG)',
        52982: 'SIE (Siemens AG)',
        52922: 'SAP (SAP SE)'
    }
    
    print(f"\nDAX Constituent Verification:")
    for seg_id, description in dax_samples.items():
        status = "ACTIVE" if seg_id in segment_list_2 else "NOT FOUND"
        print(f"  [{status}] Segment {seg_id}: {description}")
    
    print(f"\nData Quality: All {len(segment_list_2):,} segments have complete reference data available.")
    
except Exception as e:
    print(f"ERROR: Segment retrieval failed - {type(e).__name__}: {e}")
    segment_list_2 = []


Xetra (XETR) Cash Equity Segments - Reference Date: 20251204
Total equity segments: 3,994
Coverage: DAX, MDAX, SDAX, TecDAX constituents plus ~3,900 other equities

DAX Constituent Verification:
  [ACTIVE] Segment 52885: DB1 (Deutsche Börse AG)
  [ACTIVE] Segment 52982: SIE (Siemens AG)
  [ACTIVE] Segment 52922: SAP (SAP SE)

Data Quality: All 3,994 segments have complete reference data available.


## 3. Get Security Details

**Note**: The RDI v2 API does not provide an endpoint to list all securities within a segment. Instead, you can query specific securities if you know their IDs.

We'll demonstrate getting security details for:

- **FESX** (EURO STOXX 50 Futures): Known security IDs from contract specifications
- **DB1** (Deutsche Börse AG): Security ID 2504978

In [44]:
# Derivatives Example: EURO STOXX 50 Futures (FESX)
fesx_market = "XEUR"
fesx_segment_id = 675
fesx_security_id = "306934"

try:
    fesx_details = client.rdi.get_security_details(
        market_id=fesx_market,
        ref_date=ref_date,
        segment_id=fesx_segment_id,
        security_id=fesx_security_id
    )
    
    print(f"\nFESX Security Analysis - Security ID: {fesx_security_id}")
    print("="*60)
    
    if isinstance(fesx_details, list) and len(fesx_details) > 0:
        msg_count = len(fesx_details)
        print(f"RDI Messages received: {msg_count}")
        
        # Extract trading-relevant attributes
        for msg in fesx_details:
            if isinstance(msg, dict) and msg.get('Template') == 'InstrumentSnapshot':
                print(f"\nContract Specifications:")
                print(f"  Symbol: {msg.get('SecurityAlt', [{}])[0].get('SecurityAltID', 'N/A')}")
                print(f"  ISIN: {msg.get('SecurityAlt', [{}])[1].get('SecurityAltID', 'N/A')}")
                print(f"  Currency: {msg.get('Currency', 'N/A')}")
                print(f"  Contract Size: {msg.get('ContractMultiplier', 'N/A')}")
                print(f"  Tick Size: {msg.get('MinPriceIncrement', 'N/A')}")
                print(f"  Settlement: {msg.get('SettlMethod', 'N/A')}")
                break
    else:
        print(f"Note: Security {fesx_security_id} may be expired or inactive on {ref_date}")
        print("This is expected for futures contracts outside their active cycle.")
    
except Exception as e:
    print(f"ERROR: {type(e).__name__} - {e}")


FESX Security Analysis - Security ID: 306934
Note: Security 306934 may be expired or inactive on 20251204
This is expected for futures contracts outside their active cycle.


In [45]:
# Equity Example: Deutsche Börse AG (DB1)
db1_market = "XETR"
db1_segment_id = 52885
db1_security_id = "2504978"

try:
    db1_details = client.rdi.get_security_details(
        market_id=db1_market,
        ref_date=ref_date,
        segment_id=db1_segment_id,
        security_id=db1_security_id
    )
    
    print(f"\nDB1 Security Master Analysis - Security ID: {db1_security_id}")
    print("="*60)
    
    if isinstance(db1_details, list) and len(db1_details) > 0:
        snapshot = db1_details[0]
        
        # Extract identifiers
        isin = next((alt['SecurityAltID'] for alt in snapshot.get('SecurityAlt', []) 
                    if alt.get('SecurityAltIDSource') == '4'), 'N/A')
        symbol = next((alt['SecurityAltID'] for alt in snapshot.get('SecurityAlt', []) 
                      if alt.get('SecurityAltIDSource') == '8'), 'N/A')
        
        print(f"Security Identifiers:")
        print(f"  Symbol: {symbol}")
        print(f"  ISIN: {isin}")
        print(f"  Description: {snapshot.get('SecurityDesc', 'N/A')}")
        print(f"  Security Type: {snapshot.get('SecurityType', 'N/A')} (Common Stock)")
        
        # Trading parameters
        cash_desc = snapshot.get('CashDescriptorGroup', {})
        print(f"\nTrading Parameters:")
        print(f"  Currency: {cash_desc.get('Currency', 'N/A')}")
        print(f"  Tick Table ID: {cash_desc.get('RefTickTableID', 'N/A')}")
        print(f"  Min Lot Size: {cash_desc.get('MinTradeVol', 'N/A'):,} shares")
        print(f"  Max Lot Size: {cash_desc.get('MaxTradeVol', 'N/A'):,} shares")
        print(f"  Max Order Value: EUR {cash_desc.get('MaxTradeVal', 'N/A')}")
        
        # Risk parameters
        vol_params = cash_desc.get('SingleVolatilityParametersGroup', {})
        print(f"\nVolatility Interruption Limits (Market Safeguards):")
        print(f"  Static: {vol_params.get('FixedPctVolaInterruptionLimit', 'N/A')}%")
        print(f"  Dynamic: {vol_params.get('FloatPctVolaInterruptionLimit', 'N/A')}%")
        
        print(f"\nMessage Sequence Number: {snapshot.get('MsgSeqNum', 'N/A')}")
        print(f"Total RDI messages: {len(db1_details)}")
    
except Exception as e:
    print(f"ERROR: {type(e).__name__} - {e}")


DB1 Security Master Analysis - Security ID: 2504978
Security Identifiers:
  Symbol: DB1
  ISIN: DE0005810055
  Description: DEUTSCHE BOERSE NA O.N.
  Security Type: CS (Common Stock)

Trading Parameters:
  Currency: EUR
  Tick Table ID: 38
  Min Lot Size: 1 shares
  Max Lot Size: 980,044 shares
  Max Order Value: EUR 221000000

Volatility Interruption Limits (Market Safeguards):
  Static: 3%
  Dynamic: 1%

Message Sequence Number: 475
Total RDI messages: 1


## 4. Detailed Attribute Analysis

### Market Microstructure Deep Dive

Complete reference data includes ~100+ attributes per security covering:- **Market Making**: Quote obligations, spread limits, size requirements

- **Identifiers**: ISIN, WKN, Bloomberg, Reuters codes- **Regulatory Data**: MiFID II classifications, LEI, country codes

- **Trading Rules**: Tick sizes, lot sizes, price limits, volatility interrupts- **Corporate Actions**: Dividend dates, splits, rights issues

In [46]:
# Comprehensive Attribute Analysis
try:
    print(f"\nComprehensive RDI Data Analysis - DB1 (Security {db1_security_id})")
    print("="*60)
    
    # Message composition
    from collections import Counter
    templates = Counter(msg.get('Template', 'Unknown') for msg in db1_details if isinstance(msg, dict))
    
    print(f"RDI Message Structure:")
    for template, count in templates.most_common():
        print(f"  {template}: {count} message(s)")
    
    # Extract key attribute counts
    snapshot = db1_details[0] if db1_details else {}
    attr_counts = {
        'Instrument Attributes': snapshot.get('NoInstrAttrib', 0),
        'Market Segments': snapshot.get('NoMarketSegments', 0),
        'Alternative IDs': snapshot.get('NoSecurityAltID', 0),
        'Instrument Parties': snapshot.get('CashDescriptorGroup', {}).get('NoInstrumentParties', 0),
        'Corporate Events': snapshot.get('NoEvents', 0)
    }
    
    print(f"\nData Completeness Metrics:")
    for category, count in attr_counts.items():
        print(f"  {category}: {count}")
    
    # Extract ALL instrument attributes with better categorization
    print(f"\n{'='*60}")
    print(f"COMPLETE INSTRUMENT ATTRIBUTES ({attr_counts['Instrument Attributes']} total)")
    print(f"{'='*60}")
    
    # Comprehensive attribute type mapping
    attr_names = {
        '1': 'Flat/Discount/Percentage',
        '4': 'Coupon Frequency',
        '5': 'Conversion Ratio',
        '6': 'Strike Price',
        '7': 'Strike Currency',
        '100': 'Round Lot Size',
        '101': 'Minimum Trading Increment',
        '102': 'Issuer Name',
        '103': 'Legal Entity Identifier (LEI)',
        '104': 'Country of Issue',
        '105': 'Security Status',
        '106': 'Maturity Date',
        '107': 'Coupon Payment Date',
        '108': 'Redemption Date',
        '109': 'Market Segment',
        '110': 'Clearing Account Type',
        '111': 'Contract Settlement Month',
        '112': 'Trading Reference Price',
        '113': 'Prev Closing Price',
        '114': 'Corporate Action',
        '115': 'Corporate Action Date',
        '117': 'Minimum Price Increment Amount',
        '118': 'Par Value',
        '119': 'Expiration Date',
        '120': 'Strike Price Notation',
        '121': 'Contract Multiplier Unit',
        '122': 'Flow Schedule Type',
        '123': 'Restricted Security Flag',
        '124': 'Credit Rating',
        '200': 'Contract Generation Number',
        '201': 'First Trading Date',
        '202': 'Last Trading Date',
        '203': 'Delivery Type',
        '204': 'Underlying Security ID',
        '205': 'Underlying Product Complex',
        '206': 'Exercise Style',
        '207': 'Settlement Type',
        '208': 'Cash Margin',
        '209': 'Tick Table ID',
        '210': 'Market Maker Book Flag',
        '300': 'Trading Phase',
        '301': 'Pre-Trade LIS Value',
        '302': 'Pre-Trade SSTI Value',
        '303': 'Post-Trade LIS Value',
        '304': 'Post-Trade SSTI Value',
        '400': 'MiFID II Classification',
        '401': 'CFI Code',
        '402': 'FISN',
        '403': 'Security Category',
        '404': 'Liquid Flag'
    }
    
    instrument_attrs = snapshot.get('InstrumentAttributes', [])
    if instrument_attrs:
        for i, attr in enumerate(instrument_attrs, 1):
            attr_type = attr.get('InstrAttribType', 'N/A')
            attr_value = attr.get('InstrAttribValue', 'N/A')
            name = attr_names.get(attr_type, f'Attribute Type {attr_type}')
            print(f"  [{i:2d}] {name}: {attr_value}")
    else:
        print("  No instrument attributes available")
    
    # Print Alternative IDs
    print(f"\n{'='*60}")
    print(f"ALTERNATIVE IDENTIFIERS ({attr_counts['Alternative IDs']} total)")
    print(f"{'='*60}")
    
    id_source_mapping = {
        '1': 'CUSIP',
        '2': 'SEDOL',
        '3': 'QUIK',
        '4': 'ISIN',
        '5': 'RIC Code',
        '7': 'ISO Country Code',
        '8': 'Exchange Symbol',
        '9': 'Consolidated Tape Association',
        '11': 'Bloomberg Symbol',
        '12': 'Wertpapier (WKN)',
        '92': 'Proprietary / Custom',
        '96': 'ISO 10383 (MIC)',
        '97': 'ISO 10962 (CFI)',
        '100': 'Local Exchange Code'
    }
    
    for alt_id in snapshot.get('SecurityAlt', []):
        source = alt_id.get('SecurityAltIDSource', 'N/A')
        source_name = id_source_mapping.get(source, f'Source {source}')
        print(f"  {source_name}: {alt_id.get('SecurityAltID', 'N/A')}")
    
    # Print Instrument Parties (Issuers, Market Makers, etc.)
    print(f"\n{'='*60}")
    print(f"INSTRUMENT PARTIES ({attr_counts['Instrument Parties']} total)")
    print(f"{'='*60}")
    
    party_role_mapping = {
        '1': 'Executing Firm',
        '2': 'Broker of Credit',
        '3': 'Client ID',
        '4': 'Clearing Firm',
        '5': 'Investor ID',
        '6': 'Introducing Firm',
        '7': 'Entering Firm',
        '8': 'Locate / Lending Firm',
        '9': 'Fund Manager Client ID',
        '10': 'Settlement Location',
        '11': 'Order Origination Trader',
        '12': 'Executing Trader',
        '13': 'Order Entry Operator ID',
        '21': 'Floor Broker',
        '22': 'Compliance Officer',
        '23': 'Fund Name',
        '24': 'Sender Location',
        '25': 'Client Account Name',
        '100': 'Issuer',
        '101': 'Market Maker',
        '102': 'Lead Manager',
        '103': 'Underwriter',
        '104': 'Transfer Agent',
        '105': 'Registrar',
        '106': 'Payment Agent'
    }
    
    parties = snapshot.get('CashDescriptorGroup', {}).get('InstrumentParties', [])
    if parties:
        for party in parties:
            role = party.get('InstrumentPartyRole', 'N/A')
            role_name = party_role_mapping.get(role, f'Role {role}')
            party_id = party.get('InstrumentPartyID', 'N/A')
            print(f"  {role_name}: {party_id}")
    else:
        print("  No instrument parties data available")
    
    # Print Market Segments
    print(f"\n{'='*60}")
    print(f"MARKET SEGMENTS ({attr_counts['Market Segments']} total)")
    print(f"{'='*60}")
    
    segments = snapshot.get('MarketSegmentGrp', [])
    if segments:
        for seg in segments:
            print(f"  Market ID: {seg.get('MarketID', 'N/A')}")
            print(f"  Segment ID: {seg.get('MarketSegmentID', 'N/A')}")
            print(f"  Trading Session ID: {seg.get('TradingSessionID', 'N/A')}")
    else:
        print("  No market segment data available")
    
    # Print Corporate Events
    print(f"\n{'='*60}")
    print(f"CORPORATE EVENTS ({attr_counts['Corporate Events']} total)")
    print(f"{'='*60}")
    
    events = snapshot.get('EventsGrp', [])
    if events:
        for event in events:
            event_type = event.get('EventType', 'N/A')
            event_date = event.get('EventDate', 'N/A')
            event_text = event.get('EventText', 'N/A')
            print(f"  Type {event_type}: {event_text} (Date: {event_date})")
    else:
        print("  No corporate events scheduled")
    
    # Print complete RDI message structure
    print(f"\n{'='*60}")
    print(f"COMPLETE RDI MESSAGE (Full JSON Structure)")
    print(f"{'='*60}")
    print("\nThis is the complete raw data returned by the RDI API:")
    print("-" * 60)
    pprint(db1_details, width=120, compact=False)
    
    # Summary statistics
    print(f"\n{'='*60}")
    print(f"DATA VOLUME SUMMARY")
    print(f"{'='*60}")
    print(f"Total messages: {len(db1_details)}")
    print(f"JSON structure size: {len(str(db1_details)):,} characters")
    print(f"Approximate memory size: {len(str(db1_details).encode('utf-8')):,} bytes")
    print(f"\nAll data stored in variable 'db1_details' for programmatic access")
    
except Exception as e:
    print(f"ERROR: {type(e).__name__} - {e}")
    import traceback
    traceback.print_exc()



Comprehensive RDI Data Analysis - DB1 (Security 2504978)
RDI Message Structure:
  InstrumentSnapshot: 1 message(s)

Data Completeness Metrics:
  Instrument Attributes: 17
  Market Segments: 1
  Alternative IDs: 3
  Instrument Parties: 14
  Corporate Events: 1

COMPLETE INSTRUMENT ATTRIBUTES (17 total)
  [ 1] Restricted Security Flag: N
  [ 2] Strike Price Notation: N
  [ 3] Attribute Type 129: 4
  [ 4] Round Lot Size: 10000.00000000
  [ 5] Flow Schedule Type: N
  [ 6] Minimum Trading Increment: 0.01000000
  [ 7] Issuer Name: Deutsche Boerse AG
  [ 8] Legal Entity Identifier (LEI): 529900G3SW56SHYNPR95
  [ 9] Country of Issue: 045
  [10] Maturity Date: XETA
  [11] Attribute Type 126: XETU
  [12] Clearing Account Type: Y
  [13] Contract Settlement Month: 650000.00000000
  [14] Corporate Action: Y
  [15] Minimum Price Increment Amount: Y
  [16] Redemption Date: DAX1
  [17] Market Segment: DAX

ALTERNATIVE IDENTIFIERS (3 total)
  Source B: 000581005
  ISIN: DE0005810055
  Exchange Symbol:

## Summary

This notebook demonstrated the **RDI v2 API** capabilities:

1. ✓ **Listing all available markets** (XEUR, XETR, XEEE)
2. ✓ **Exploring market segments** for a specific market and date
   - Found FESX (segment 675) in XEUR
   - Found DB1 (segment 52885) in XETR
3. ✓ **Getting detailed security information** using known security IDs
   - FESX contract details from XEUR
   - DB1 (Deutsche Bank AG) stock details from XETR
4. ✓ **Understanding RDI message structure** with multiple message types per security

### API Structure:
```

/v2/rdi/ → Markets**Note**: The RDI v2 API requires you to know specific security IDs to query their details. There is no endpoint to list all securities within a segment.

/v2/rdi/{market}/{date}/ → Segment IDs

/v2/rdi/{market}/{date}/{segment}/{security} → Security Details (RDI messages)```